In [56]:
import requests
from bs4 import BeautifulSoup
from secrets import username, password

class FaceBookBot():
    login_basic_url = 'https://mbasic.facebook.com/login'
    login_mobile_url = 'https://m.facebook.com/login'
    payload = {
            'email': username,
            'pass': password
        }
    post_ID = ""

    def parse_html(self, request_url):
        with requests.Session() as session:
            post = session.post(self.login_basic_url, data=self.payload)
            parsed_html = session.get(request_url)
        return parsed_html

    def post_content(self):
        REQUEST_URL = f'https://mbasic.facebook.com/story.php?story_fbid={self.post_ID}&id=415518858611168'
        
        soup = BeautifulSoup(self.parse_html(REQUEST_URL).content, "html.parser")
        content = soup.find_all('p')
        post_content = []
        for lines in content:
            post_content.append(lines.text)
        
        post_content = ' '.join(post_content)    
        return post_content

    def date_posted(self):
        REQUEST_URL = f'https://mbasic.facebook.com/story.php?story_fbid={self.post_ID}&id=415518858611168'
        
        soup = BeautifulSoup(self.parse_html(REQUEST_URL).content, "html.parser")
        date_posted = soup.find('abbr')
        return date_posted.text

    def post_likes(self):
        limit = 200
        REQUEST_URL = f'https://mbasic.facebook.com/ufi/reaction/profile/browser/fetch/?limit={limit}&total_count=17&ft_ent_identifier={self.post_ID}'

        soup = BeautifulSoup(self.parse_html(REQUEST_URL).content, "html.parser")
        names = soup.find_all('h3')
        people_who_liked = []
        for name in names:
            people_who_liked.append(name.text)
        people_who_liked = [i for i in people_who_liked if i] 
        return people_who_liked

    def post_shares(self):        
        REQUEST_URL = f'https://m.facebook.com/browse/shares?id={self.post_ID}'
        
        with requests.Session() as session:
            post = session.post(self.login_mobile_url, data=self.payload)
            parsed_html = session.get(REQUEST_URL)
        
        soup = BeautifulSoup(parsed_html.content, "html.parser")
        names = soup.find_all('span')
        people_who_shared = []
        for name in names:
            people_who_shared.append(name.text)
        return people_who_shared
    
    def get_event(self, event_ID):
        REQUEST_URL = f'https://m.facebook.com/events/{event_ID}'
        
        soup = BeautifulSoup(self.parse_html(REQUEST_URL).content, "html.parser")
        output = {}
        title = soup.find('h1', class_='by')
        output['title'] = title.get_text()
#         time_location = soup.find_all('table', class_='_4g33')
        
        time_location = soup.find_all('div', class_='cq cr x')
        output['time'] = time_location[0].get_text()
        location = soup.find_all('div', class_='cs ct x')[1]
        output['location'] = time_location[1].get_text() + ': ' + location.get_text()
        description = soup.find('div', class_='_52ja _2pi9 _2pip _2s23')
        output['description'] = description.get_text()
        responses = soup.find_all('a', class_='_2058')
        output['going'] = responses[0].get_text()
        output['interested'] = responses[1].get_text()
        shares = soup.find_all('div', class_='_2058')[1]
        output['shares'] = shares.get_text()
        return output

In [57]:
bot = FaceBookBot()
event_ID = 2842763029277309
print(bot.get_event(event_ID))

{'title': 'Pan-Asian Presents: Dreamscape', 'time': 'Friday, April 9, 2021 at 9:00 PM – 11:00 PM EDT', 'location': 'Online Event: https://www.youtube.com/watch?v=J_-2Zoo_cNg&ab_channel=Pan-AsianDanceTroupe', 'description': 'Pan-Asian Dance Troupe presents our 20th Annual Show, Dreamscape.  Where do our dreams take us when we fall asleep? Follow Pan-Asian as we explore the mystifying landscape of our subconscious imagination.  FRIDAY, APRIL 9TH, 9PM EDT LIVE ON YOUTUBE: https://youtu.be/J_-2Zoo_cNg  Check out https://www.panasiandance.com/dreamscape21 for more info!  ______________________________________________________ <---AD SUBMISSIONS CLOSED---> Want to support your favorite PADT member? Submit an ad for your friends!  Size/Price Info: · Full-Page (5½” x 8½”) : $15 · Half-Page (5½” x 4¼”) : $10 · Quarter-Page (2¾” x 4¼”) : $5 · Shoutouts (100 characters): $2 Please submit an electronic version of the ad either in PDF or .jpg form with the name “To_[name]_From_[name]_[SIZE].” to pan